In [54]:
# https://towardsdatascience.com/combining-numerical-and-text-features-in-deep-neural-networks-e91f0237eea4
"""combine meta data with text in keras"""

'combine meta data with text in keras'

In [1]:
import spacy
import gensim
nlp = spacy.load('en')
import pandas as pd
import numpy as np
import re
import demoji

In [2]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/constraint_Hindi_Train - Sheet1.csv"
data = pd.read_csv(file_path)
data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [14]:
# making labels
from sklearn.preprocessing import MultiLabelBinarizer

y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':2,'fake':3,'defamation':4}
for labels in data['Labels Set']:
    temp = list(set([str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    #temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y = [[int(x) for x in xx] for xx in y]
print(y[:10])
y = MultiLabelBinarizer().fit_transform(y)
y[:10]

[[1, 2], [0], [0], [2, 4], [0], [3], [0], [0], [0], [0]]


array([[0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [4]:
def hindi_tweet_cleaning(tweet):
    tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
    pat = re.compile(r"([',./@#(;_`:|\-)!])")
    tweet = pat.sub("", tweet)
    tweet_split_word_list = tweet.split()
    tweet_split_word_list_new = []
    #print(tweet_split_word_list)
    for word in tweet_split_word_list:
        if not word.isalpha():
            tweet_split_word_list_new.append(word)
    tweet = " ".join(tweet_split_word_list_new)
    tweet = demoji.replace(tweet,"")
    tweet = tweet.rstrip("\n")
    tweet = re.sub('\n', '', tweet)
    tweet = tweet.replace('\u200b', '')
    tweet = tweet.replace('\u200c', '')
    tweet = tweet.replace('\u200d', '')
    return tweet

In [5]:
all_tweets= []
for tweet in data['Post']:
    all_tweets.append(hindi_tweet_cleaning(tweet))

In [6]:
embedding_loc = "/home/ayush/Documents/personal/research/aaai/hostility_classification/fastText/cc.hi.300.bin"

In [7]:
import fasttext
import fasttext.util
ft = fasttext.load_model(embedding_loc)

In [8]:
X = []
for tweet in all_tweets:
    X.append(ft.get_sentence_vector(tweet))

In [9]:
X_train = X[:5500]
y_train = y[:5500]

X_test = X[5500:]
y_test = y[5500:]

In [16]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, multi_class='ovr',n_jobs= -1).fit(X_train, y_train)

ValueError: y should be a 1d array, got an array of shape (5500, 5) instead.

In [65]:
clf = OneVsRestClassifier(SVC())

In [66]:
clf.fit(X_train,y_train)

OneVsRestClassifier(estimator=SVC())

In [67]:
predictions=clf.predict(X_test)

In [68]:
from sklearn.metrics import accuracy_score

In [69]:
accuracy_score(y_test,predictions)

0.4780701754385965

In [72]:
len(y_test)

228

In [71]:
from sklearn import metrics
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.86      0.89      0.87       117
           1       1.00      0.02      0.05        42
           2       0.67      0.15      0.25        26
           3       1.00      0.05      0.09        42
           4       0.00      0.00      0.00        18

   micro avg       0.85      0.45      0.59       245
   macro avg       0.71      0.22      0.25       245
weighted avg       0.82      0.45      0.47       245
 samples avg       0.48      0.48      0.48       245



/home/ayush/anaconda3/envs/personal/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ayush/anaconda3/envs/personal/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
# testing data

In [73]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/Constraint_Hindi_Valid - Sheet1.csv"
data = pd.read_csv(file_path)

In [74]:
# making labels
from sklearn.preprocessing import MultiLabelBinarizer

y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':2,'fake':3,'defamation':4}
for labels in data['Labels Set']:
    temp = list(set([str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    #temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y = [[int(x) for x in xx] for xx in y]
print(y[:10])
y = MultiLabelBinarizer().fit_transform(y)
y[:10]

[[0], [4], [0], [0], [0], [0], [0], [3], [0], [0]]


array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [75]:
all_tweets= []
for tweet in data['Post']:
    all_tweets.append(hindi_tweet_cleaning(tweet))

In [76]:
X = []
for tweet in all_tweets:
    X.append(ft.get_sentence_vector(tweet))

In [77]:
predictions=clf.predict(X)

In [78]:
accuracy_score(y,predictions)

0.5067817509247842

In [79]:
from sklearn import metrics
print(metrics.classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       435
           1       1.00      0.02      0.04       110
           2       0.73      0.21      0.33       103
           3       0.91      0.06      0.12       160
           4       0.00      0.00      0.00        77

   micro avg       0.86      0.48      0.62       885
   macro avg       0.70      0.24      0.27       885
weighted avg       0.80      0.48      0.50       885
 samples avg       0.52      0.51      0.52       885



/home/ayush/anaconda3/envs/personal/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ayush/anaconda3/envs/personal/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
